# 下载视频

In [24]:
import json
import os
import random
import re
import requests
import time
import execjs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import csv

base_url = 'https://www.xiaohongshu.com/user/profile/'
more_url = 'https://edith.xiaohongshu.com/api/sns/web/v1/user_posted'
feed_url = 'https://edith.xiaohongshu.com/api/sns/web/v1/feed'
img_url = 'https://sns-img-bd.xhscdn.com/'
video_url = 'https://sns-video-bd.xhscdn.com/'
js = execjs.compile(open(r'./info.js', 'r', encoding='utf-8').read())
if not os.path.exists("datas"):
    os.mkdir("datas")
class Bro:
    base_url = "https://www.xiaohongshu.com/user/profile/6185ce66000000001000705b"
    option = ChromeOptions()
    option.add_experimental_option('excludeSwitches', ['enable-automation'])
    # option.add_argument('--headless')
    # option.add_argument('--disable-gpu')
    option.add_argument("--disable-blink-features=AutomationControlled")
    option.add_argument('--start-maximized')
    option.add_argument('window-size=1920x1080')
    option.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36")
    s = Service(r"/Users/tianyc/Documents/python/xiaohongshu/Spider_XHS-master/chromedriver")

    def get_cookie(self):
        self.bro = webdriver.Chrome(service=self.s, options=self.option, )
        self.bro.implicitly_wait(10)
        self.bro.get(self.base_url)
        self.wait = WebDriverWait(self.bro, 3)
        try:
            me = self.wait.until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/div[1]/ul/li[4]/div/a')))
            print("已登录， 直接获取cookie")
        except:
            print("需要登录")
            time.sleep(1)
            print('正在点击主页登录按钮')
            login = self.wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/div[1]/ul/div[1]/button')))
            login.click()
            time.sleep(1)
            login_block = self.wait.until(EC.presence_of_element_located(
                (By.XPATH, '/html/body/div[1]/div[1]/div/div[1]')))
            while True:
                try:
                    time.sleep(5)
                    print('正在尝试登录')
                    if self.bro.current_url.find('https://www.xiaohongshu.com/') != -1:
                        if not login_block.is_displayed():
                            print('登录成功')
                    else:
                        continue
                except:
                    print('登录成功')
                    break

        print('正在获取cookie')
        time.sleep(5)
        cookies_ = self.bro.get_cookies()
        cookies = {}
        for cookie in cookies_:
            cookies[cookie['name']] = cookie['value']
        self.bro.quit()
        with open("cookies.txt", "w", encoding="utf-8") as f:
            f.write(str(cookies))
        return cookies
        
def download_media(path, name, img_url_id, type, fm=False):
    media_url = img_url + img_url_id if type == 'image' else video_url + img_url_id
    if type == 'image':
        print("封面开始下载") if fm else print(f"第{name}个图片开始下载")
        content = requests.get(media_url).content
        with open(path + '/' + name + '.jpg', mode="wb") as f:
            f.write(content)
            print("封面下载完成") if fm else print(f"第{name}个图片下载完成")
    elif type == 'video':
        # 将视频URL保存到CSV文件中
        csv_file_path = path + '/videos.csv'
        with open(csv_file_path, mode='a', encoding='utf-8', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([name, media_url])
        print(f"{name}保存到CSV文件中")   

def timestamp_to_time(timestamp):
    time_local = time.localtime(timestamp / 1000)
    dt = time.strftime("%Y年%m月%d日%H点%M分%S秒", time_local)
    return dt

def checkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

def save_detail(path, detail):
    with open(path + '/' + 'detail.txt', mode="w", encoding="utf-8") as f:
        # 逐行输出到txt里
        f.write(f"用户名: {detail['user_nickname']}\n")
        f.write(f"笔记标题: {detail['note_title']}\n")
        f.write(f"笔记描述: {detail['note_desc']}\n")
        f.write(f"笔记时间: {detail['note_time']}\n")
        f.write(f"笔记地点: {detail['note_ipLocation']}\n")

cookies = {
    "xsecappid": "",
    "a1": "",
    "webId": "",
    "gid": "",
    "webBuild": "3.3.4",
    "web_session": "",
    "websectiga": "",
    "sec_poison_id": ""
}
headers = {
    "authority": "edith.xiaohongshu.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "content-type": "application/json;charset=UTF-8",
    "origin": "https://www.xiaohongshu.com",
    "referer": "https://www.xiaohongshu.com/",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.188",
    "x-s": "",
    "x-t": ""
}
params = {
    "num": "30",
    "cursor": "",
    "user_id": "",
    "image_scenes": ""
}

def check_cookie():
    global cookies
    try:
        if not os.path.exists("cookies.txt"):
            raise Exception("获取cookie")
        test_user_id = '5ad2ede14eacab146f865fe9'
        with open("cookies.txt", "r", encoding="utf-8") as f:
            cookies_obj = f.read()
        cookies_local = eval(cookies_obj)
        params['user_id'] = test_user_id
        params['cursor'] = ''
        api = f"/api/sns/web/v1/user_posted?num=30&cursor=&user_id={test_user_id}&image_scenes="
        ret = js.call('get_xs', api, '', cookies_local['a1'])
        headers['x-s'], headers['x-t'] = ret['X-s'], str(ret['X-t'])
        response = requests.get(more_url, headers=headers, cookies=cookies_local, params=params)
        res = response.json()
        if not res["success"]:
            raise Exception("cookie失效")
        else:
            cookies = cookies_local
            print("cookie有效")
    except:
        print("cookie失效，正在重新获取")
        bro = Bro()
        cookies_local = bro.get_cookie()
        if cookies_local:
            print('cookies获取成功')
        else:
            print('cookies获取失败')
        cookies = cookies_local
def handle_note(note_id):
    data = {
        "source_note_id": note_id
    }
    data = json.dumps(data, separators=(',', ':'))
    ret = js.call('get_xs', '/api/sns/web/v1/feed', note_id, cookies['a1'])
    headers['x-s'], headers['x-t'] = ret['X-s'], str(ret['X-t'])
    response = requests.post(feed_url, headers=headers, cookies=cookies, data=data)
    res = response.json()
    data = res['data']['items'][0]
    user_dir_name = re.sub(r'[\\/:*?"<>|]', '', data['note_card']['user']['nickname'])
    user = {
        "nick_name": data['note_card']['user']['nickname'],
        "user_id": data['note_card']['user']['user_id'],
        "avatar": data['note_card']['user']['avatar'],
        "user_dir_name": user_dir_name
    }
    note_title = data['note_card']['title']
    print(f'用户: {user["nick_name"]}， 笔记标题: {note_title} 开始下载')
    note_type = data['note_card']['type']
    note_time = data['note_card']['time']
    if 'ip_location' in data['note_card']:
        note_ip_location = data['note_card']['ip_location']
    else:
        note_ip_location = '未知'
    note_desc = data['note_card']['desc']
    note_file_name = re.sub(r'[\\/:*?"<>|]', '', note_title)
    file_path = f"./datas/video/{note_id}"
    checkdir(file_path)

    if note_type == 'normal':
        for index, img in enumerate(data['note_card']['image_list']):
            img_url_id = img['trace_id']
            download_media(file_path, str(index), img_url_id, 'image')

    elif note_type == 'video':
        img_url_id = data['note_card']['image_list'][0]['trace_id']
        download_media(file_path, '封面', img_url_id, 'image', fm=True)
        video_url_id = data['note_card']['video']['consumer']['origin_video_key']
        download_media(file_path, '视频', video_url_id, 'video')
    detail = {
        "user_nickname": user['nick_name'],
        "note_title": note_title,
        "note_desc": note_desc,
        "note_time": timestamp_to_time(int(note_time)),
        "note_ipLocation": note_ip_location,

    }
    save_detail(file_path, detail)
    print(f'用户: {user["nick_name"]}， 笔记标题: {note_title} 下载完成')
    print('--------------------------------------------------------------------------------')

def main():
    check_cookie()
    # https://www.xiaohongshu.com/explore/           64d06670000000000800fb4a    详细页获取
    note_id_list = [


'64d0c69f000000001201847f',
'64d0c609000000001201b4ba',
'64d0b761000000001201b7de',
'64d07d01000000000c03640e',
'64cfd481000000001201eefb',
'64cfd1ef000000000a01aaa0',
'64cfd092000000000103da1d',
'64cfc612000000001700eff3',
'64cfaa35000000000103d8c9',
'64cfa9e1000000000a019b5a',
'64cfa888000000000c035408',
'64cf99ad000000000b02b21d',
'64cf8d01000000000103c425',
'64cf5d3d0000000010032c0d',
'64cf51e5000000000800fc05',
'64cf23d900000000170193ab',
'64cef8520000000017018f25',
'64cee143000000000c0354e1',
'64cea3c3000000000b02bea5',
'64ce5098000000001700ea11',
'64ce37ab000000001201b5a7',
'64ce32e9000000000a018467',
'64ce31bb000000000c036d1d',
'64cded08000000000800f075',
'64cde4c6000000000c034042',
'64cd4f7a000000001201a887',
'64cd2d4a00000000100318b4',
'64cd2aeb000000001700c352',
'64ccdef6000000000c036bf9',
'64cce95c000000000103d675',
'64ccd976000000000a01b839',
'64ccc309000000000a01b5b2',
'64cc8eab000000001700cbfa',
'64cc4e74000000000800f553',
'64cbdf47000000000b029715',
'64cbd0960000000010032bb5',
'64cbae860000000010030cda',
'64cb714e000000000103df0a',
'64cb6d41000000000b0290f5',
'64cb67cf000000000103fc66',
'64cb3714000000000103d3c3',
'64ca9238000000000b02a7e3',
'64ca5e49000000000a01bf80',
'64ca5224000000000a018b42',
'64ca50dd000000000c035b7c',
'64ca4cb3000000000c037322',
'64ca4b0c000000000b02b878',
'64ca47b30000000017019efc',
'64ca24b9000000000a0191a2',
'64ca18f3000000001201bbfb',
'64c9f42a000000000b029503',
'64c9c67d000000000103e4a2',
'64c9c540000000000c0351be',
'64c9b7790000000012019dfe',
'64c93d1e000000001201dfb2',
'64c92dfc000000000c03650b',
'64c92d30000000000c03550e',
'64c9146c000000000103d8c7',
'64c9127a000000000103c34d',
'64c90375000000001201dcb7',
'64c8f62b000000001201ab07',
'64c8c5e3000000001201d05a',
'64c8a56b000000000c037033',
'64c86fb6000000000103eaa3',
'64c7f228000000001201ab20',
'64c7e3f0000000000800dab3',
'64c7dca30000000017018489',
'64c7d1ff0000000012018fb0',
'64c7b4a1000000000103c685',
'64c7a44e000000001700d5e4',
'64c6fc6a000000000b029cd0',
'64c66015000000000c0352cc',
'64c65c65000000001700e57b',
'64c633fd000000001700dad6',
'64c64c1f000000001201cef1',
'64c635630000000012018297',
'64c60555000000001701b9bf',
'64c5f2ee000000000c036575',
'64c5e772000000000103d4b3',
'64c5c377000000001201d208',
'64c54bba000000000b029b1c',
'64c54a60000000001701a37b',
'64c52f11000000001201d159',
'64c52573000000001201ad16',
'64c507e4000000000a01bb0f',
'64c5051d000000000a0193ff',
'64c3ff4d000000000a018109',
'64c4d8c0000000000800fdf7',
'64c4d0530000000012019b82',
'64c4a0fb000000001201a386',
'64c49a2a000000000a01a2f4',
'64c4867e000000000b02bc2d',
'64c3d7530000000012018786',
'64c445d9000000000b0282ec',
'64c4087b000000000c037a25',
'64c3f093000000001700dc80',
'64c3db58000000000a019513',
'64c3d94b000000001701ae3e',
'64c3c70f000000001700d976',
'64c3c09b000000000800c5c8',
'64c3c035000000001201c4a3',
'64c3a490000000000103dc09',
'64c375fb0000000012018143',
'64c36c1d000000000a018c21',
'64c34dc8000000001201ba99',
'64c247f1000000000b02b211',
'64c33765000000001700cc80',
'64c29bbc000000001701a738',
'64c29bb0000000001201ddbb',
'64c2830a000000001201d607',
'64c27e9b000000000a01a777',
'64c277db000000001701816c',
'64c26727000000000a018b62',
'64c253d3000000001201d831',
'64c251a5000000000800dd9c',
'64c2381e0000000010030571',
'64c22f34000000000103ce0a',
'64c2280e000000000800d69d',
'64c21215000000001201c8c2',
'64c1a81a000000001201f2b5',
'64c147cd000000000800cdef',
'64c13eb1000000000800e6d4',
'64c13cf8000000000c034da7',
'64c12b39000000000c0348ee',
'64c12156000000000800fafb',
'64c11282000000000800ebf0',
'64c0f0cb000000001700c73c',
'64c0f007000000000103d721',
'64c0ad7a000000001701ad51',
'64c073a5000000000c0370c4',
'64bffe6f0000000010031455',
'64bfe2b1000000000800dad8',
'64bfe21e000000001201e97c',
'64bfe146000000000c03726c',
'64bfdeb700000000120197a0',
'64bfbead000000000a01a43e',
'64bfa10500000000170194ef',
'64bf9e5a000000001201835c',
'64bf9c56000000001201c5e6',
'64bf903b000000000a01a794',
'64bf5fe5000000000800f04f',
'64bef7d3000000001700e116',
'64be8de0000000000a018ed1',
'64be7d61000000000a018f7b',
'64be694e000000000a018ecc',
'64be63fa000000001201ea70',
'64be581a000000000800fa05',
'64be4a8c000000000c034fd6',
'64be3c8a000000001701995d',
'64be38b8000000001201811d',
'64be2b93000000000103c0d0',
'64be201d00000000100334db',
'64be06aa00000000120183a2',
'64be01f0000000000b02936a',
'64bd8314000000000b02aad7',
'64bd5dbc000000000c03594b',
'64bd3f72000000001201e652',
'64bd3d1d0000000010033f39',
'64bd3524000000000103e7d7',
'64bd3202000000000800ec2f',
'64bcc1f60000000012019e89',
'64bd0732000000001700c423',
    ]
    for note_id in note_id_list:
        try:
            handle_note(note_id)
        except TypeError as e:
            print(f"处理note_id {note_id} 出错: {e}")
            continue
            



if __name__ == '__main__':
    main()

cookie有效
用户: 小邢聊欧洲， 笔记标题: 希腊欧债危机，难民问题，就业危机...... 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 小邢聊欧洲， 笔记标题: 希腊欧债危机，难民问题，就业危机...... 下载完成
--------------------------------------------------------------------------------
用户: 陈菲|生命探索发展教练💫， 笔记标题: ICF核心能力：如何成为更专业的教练 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 陈菲|生命探索发展教练💫， 笔记标题: ICF核心能力：如何成为更专业的教练 下载完成
--------------------------------------------------------------------------------
用户: 杰杰琦琦的妈妈， 笔记标题: 如何选择任务？ 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 杰杰琦琦的妈妈， 笔记标题: 如何选择任务？ 下载完成
--------------------------------------------------------------------------------
用户: 咨询师解心， 笔记标题: 裸眼3D+元宇宙咨询，距离数字游民更近一步～ 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 咨询师解心， 笔记标题: 裸眼3D+元宇宙咨询，距离数字游民更近一步～ 下载完成
--------------------------------------------------------------------------------
用户: 范范范大力， 笔记标题: 我的办公室里有绝美的风景和好喝的咖啡 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 范范范大力， 笔记标题: 我的办公室里有绝美的风景和好喝的咖啡 下载完成
--------------------------------------------------------------------------------
用户: 旅行家二洁， 笔记标题: 在机场玩滑板谁懂这种丝滑啊！！ 开

用户: 北大台灣學姐伊蓮 Elaine， 笔记标题: vlog丨粉丝数慢慢在增加，能有妳的一份吗 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 北大台灣學姐伊蓮 Elaine， 笔记标题: vlog丨粉丝数慢慢在增加，能有妳的一份吗 下载完成
--------------------------------------------------------------------------------
用户: 讲故事的Miya， 笔记标题: 努力工作，是为了成为“自由的灵魂” 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 讲故事的Miya， 笔记标题: 努力工作，是为了成为“自由的灵魂” 下载完成
--------------------------------------------------------------------------------
用户: 微拉vera， 笔记标题: INFP真的会被work from home治愈啊！ 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 微拉vera， 笔记标题: INFP真的会被work from home治愈啊！ 下载完成
--------------------------------------------------------------------------------
用户: 杰杰琦琦的妈妈， 笔记标题: 工作日常 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 杰杰琦琦的妈妈， 笔记标题: 工作日常 下载完成
--------------------------------------------------------------------------------
用户: TiTi syu， 笔记标题: 独旅❤️巴厘岛| 乌鲁瓦图悬崖网红酒吧savaya 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: TiTi syu， 笔记标题: 独旅❤️巴厘岛| 乌鲁瓦图悬崖网红酒吧savaya 下载完成
--------------------------------------------------------------------------------
用户: TiTi sy

用户: BZhang， 笔记标题: 此生无憾 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: BZhang， 笔记标题: 此生无憾 下载完成
--------------------------------------------------------------------------------
用户: 蓝头小萌， 笔记标题: 自由职业之后，感觉格局都打开了 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 蓝头小萌， 笔记标题: 自由职业之后，感觉格局都打开了 下载完成
--------------------------------------------------------------------------------
用户: Daisy， 笔记标题: 如何在不确定的旅居生活中找确定性 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: Daisy， 笔记标题: 如何在不确定的旅居生活中找确定性 下载完成
--------------------------------------------------------------------------------
用户: 请大喊三声如意， 笔记标题: 怎么还有人不知道西雅图的百人户外瑜伽 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 请大喊三声如意， 笔记标题: 怎么还有人不知道西雅图的百人户外瑜伽 下载完成
--------------------------------------------------------------------------------
用户: 苏我入鹿， 笔记标题: 摆地摊是一种道德上的正义 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 苏我入鹿， 笔记标题: 摆地摊是一种道德上的正义 下载完成
--------------------------------------------------------------------------------
用户: 孤独的阅读者书坊， 笔记标题: 惊呆了！新西兰打工旅行这么有趣 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 孤独的阅读者书坊， 笔记标题: 惊呆了！新西兰打工旅行这么有趣 

用户: 沈清， 笔记标题: 朋友说我不适合去马尔代夫做民宿， 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 沈清， 笔记标题: 朋友说我不适合去马尔代夫做民宿， 下载完成
--------------------------------------------------------------------------------
用户: 爱游戏的椒胡（全球旅居中）， 笔记标题: 我的独立游戏《小生活》推出了全新烧糊系统 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 爱游戏的椒胡（全球旅居中）， 笔记标题: 我的独立游戏《小生活》推出了全新烧糊系统 下载完成
--------------------------------------------------------------------------------
用户: 杰杰琦琦的妈妈， 笔记标题: 没有电脑可以吗？ 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 杰杰琦琦的妈妈， 笔记标题: 没有电脑可以吗？ 下载完成
--------------------------------------------------------------------------------
用户: 二月星儿， 笔记标题: 退休倒计时1000天，远程不打卡的工作哪里找 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: 二月星儿， 笔记标题: 退休倒计时1000天，远程不打卡的工作哪里找 下载完成
--------------------------------------------------------------------------------
用户: Susan， 笔记标题: 5个在家搞钱平台｜赚美金、不会英语、全球旅居 开始下载
封面开始下载
封面下载完成
视频保存到CSV文件中
用户: Susan， 笔记标题: 5个在家搞钱平台｜赚美金、不会英语、全球旅居 下载完成
--------------------------------------------------------------------------------
用户: 杰杰琦琦的妈妈， 笔记标题: 今日工作工作总结 开始下载
封面开始下载
封面下载完成


# 下载图片

In [1]:
import json
import os
import random
import re
import requests
import time
import execjs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait


base_url = 'https://www.xiaohongshu.com/user/profile/'
more_url = 'https://edith.xiaohongshu.com/api/sns/web/v1/user_posted'
feed_url = 'https://edith.xiaohongshu.com/api/sns/web/v1/feed'
img_url = 'https://sns-img-bd.xhscdn.com/'
video_url = 'https://sns-video-bd.xhscdn.com/'
js = execjs.compile(open(r'./info.js', 'r', encoding='utf-8').read())
if not os.path.exists("datas"):
    os.mkdir("datas")
class Bro:
    base_url = "https://www.xiaohongshu.com/user/profile/6185ce66000000001000705b"
    option = ChromeOptions()
    option.add_experimental_option('excludeSwitches', ['enable-automation'])
    # option.add_argument('--headless')
    # option.add_argument('--disable-gpu')
    option.add_argument("--disable-blink-features=AutomationControlled")
    option.add_argument('--start-maximized')
    option.add_argument('window-size=1920x1080')
    option.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36")
    s = Service(r"/Users/tianyc/Documents/python/xiaohongshu/Spider_XHS-master/chromedriver")

    def get_cookie(self):
        self.bro = webdriver.Chrome(service=self.s, options=self.option, )
        self.bro.implicitly_wait(10)
        self.bro.get(self.base_url)
        self.wait = WebDriverWait(self.bro, 3)
        try:
            me = self.wait.until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/div[1]/ul/li[4]/div/a')))
            print("已登录， 直接获取cookie")
        except:
            print("需要登录")
            time.sleep(random.uniform(1, 3)) 
            print('正在点击主页登录按钮')
            login = self.wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/div[1]/ul/div[1]/button')))
            login.click()
            time.sleep(random.uniform(1, 3)) 
            login_block = self.wait.until(EC.presence_of_element_located(
                (By.XPATH, '/html/body/div[1]/div[1]/div/div[1]')))
            while True:
                try:
                    time.sleep(random.uniform(4, 6))
                    print('正在尝试登录')
                    if self.bro.current_url.find('https://www.xiaohongshu.com/') != -1:
                        if not login_block.is_displayed():
                            print('登录成功')
                    else:
                        continue
                except:
                    print('登录成功')
                    break

        print('正在获取cookie')
        time.sleep(random.uniform(4, 7))
        cookies_ = self.bro.get_cookies()
        cookies = {}
        for cookie in cookies_:
            cookies[cookie['name']] = cookie['value']
        self.bro.quit()
        with open("cookies.txt", "w", encoding="utf-8") as f:
            f.write(str(cookies))
        return cookies
        
def download_media(path, name, img_url_id, type, fm=False):
    media_url = img_url + img_url_id if type == 'image' else video_url + img_url_id
    if type == 'image':
        print("封面开始下载") if fm else print(f"第{name}个图片开始下载")
        content = requests.get(media_url).content
        with open(path + '/' + name + '.jpg', mode="wb") as f:
            f.write(content)
            print("封面下载完成") if fm else print(f"第{name}个图片下载完成")
    elif type == 'video':
        print(f"{name}开始下载, {media_url}")
        content = requests.get(media_url).content
        with open(path + '/' + name + '.mp4', mode="wb") as f:
            f.write(content)
            print(f"{name}下载完成")

def timestamp_to_time(timestamp):
    time_local = time.localtime(timestamp / 1000)
    dt = time.strftime("%Y年%m月%d日%H点%M分%S秒", time_local)
    return dt

def checkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

def save_detail(path, detail):
    with open(path + '/' + 'detail.txt', mode="w", encoding="utf-8") as f:
        # 逐行输出到txt里
        f.write(f"用户名: {detail['user_nickname']}\n")
        f.write(f"笔记标题: {detail['note_title']}\n")
        f.write(f"笔记描述: {detail['note_desc']}\n")
        f.write(f"笔记时间: {detail['note_time']}\n")
        f.write(f"笔记地点: {detail['note_ipLocation']}\n")

cookies = {
    "xsecappid": "",
    "a1": "",
    "webId": "",
    "gid": "",
    "webBuild": "3.3.4",
    "web_session": "",
    "websectiga": "",
    "sec_poison_id": ""
}
headers = {
    "authority": "edith.xiaohongshu.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "content-type": "application/json;charset=UTF-8",
    "origin": "https://www.xiaohongshu.com",
    "referer": "https://www.xiaohongshu.com/",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.188",
    "x-s": "",
    "x-t": ""
}
params = {
    "num": "30",
    "cursor": "",
    "user_id": "",
    "image_scenes": ""
}

def check_cookie():
    global cookies
    try:
        if not os.path.exists("cookies.txt"):
            raise Exception("获取cookie")
        test_user_id = '5ad2ede14eacab146f865fe9'
        with open("cookies.txt", "r", encoding="utf-8") as f:
            cookies_obj = f.read()
        cookies_local = eval(cookies_obj)
        params['user_id'] = test_user_id
        params['cursor'] = ''
        api = f"/api/sns/web/v1/user_posted?num=30&cursor=&user_id={test_user_id}&image_scenes="
        ret = js.call('get_xs', api, '', cookies_local['a1'])
        headers['x-s'], headers['x-t'] = ret['X-s'], str(ret['X-t'])
        response = requests.get(more_url, headers=headers, cookies=cookies_local, params=params)
        res = response.json()
        if not res["success"]:
            raise Exception("cookie失效")
        else:
            cookies = cookies_local
            print("cookie有效")
    except:
        print("cookie失效，正在重新获取")
        bro = Bro()
        cookies_local = bro.get_cookie()
        if cookies_local:
            print('cookies获取成功')
        else:
            print('cookies获取失败')
        cookies = cookies_local
        time.sleep(random.uniform(2, 4))
def handle_note(note_id):
    data = {
        "source_note_id": note_id
    }
    data = json.dumps(data, separators=(',', ':'))
    ret = js.call('get_xs', '/api/sns/web/v1/feed', note_id, cookies['a1'])
    headers['x-s'], headers['x-t'] = ret['X-s'], str(ret['X-t'])
    response = requests.post(feed_url, headers=headers, cookies=cookies, data=data)
    res = response.json()
    data = res['data']['items'][0]
    user_dir_name = re.sub(r'[\\/:*?"<>|]', '', data['note_card']['user']['nickname'])
    user = {
        "nick_name": data['note_card']['user']['nickname'],
        "user_id": data['note_card']['user']['user_id'],
        "avatar": data['note_card']['user']['avatar'],
        "user_dir_name": user_dir_name
    }
    note_title = data['note_card']['title']
    print(f'用户: {user["nick_name"]}， 笔记标题: {note_title} 开始下载')
    note_type = data['note_card']['type']
    note_time = data['note_card']['time']
    if 'ip_location' in data['note_card']:
        note_ip_location = data['note_card']['ip_location']
    else:
        note_ip_location = '未知'
    note_desc = data['note_card']['desc']
    note_file_name = re.sub(r'[\\/:*?"<>|]', '', note_title)
    file_path = f"./datas/image/{note_id}"
    checkdir(file_path)

    if note_type == 'normal':
        for index, img in enumerate(data['note_card']['image_list']):
            img_url_id = img['trace_id']
            download_media(file_path, str(index), img_url_id, 'image')

    elif note_type == 'video':
        img_url_id = data['note_card']['image_list'][0]['trace_id']
        download_media(file_path, '封面', img_url_id, 'image', fm=True)
        video_url_id = data['note_card']['video']['consumer']['origin_video_key']
        download_media(file_path, '视频', video_url_id, 'video')
    detail = {
        "user_nickname": user['nick_name'],
        "note_title": note_title,
        "note_desc": note_desc,
        "note_time": timestamp_to_time(int(note_time)),
        "note_ipLocation": note_ip_location,

    }
    save_detail(file_path, detail)
    print(f'用户: {user["nick_name"]}， 笔记标题: {note_title} 下载完成')
    print('--------------------------------------------------------------------------------')


def main():
    check_cookie()
    # https://www.xiaohongshu.com/explore/           64d06670000000000800fb4a    详细页获取
    note_id_list = [
'64f1c0ba000000001f03a3a6',
'64f1b21f000000001e02c0cc',
'64f19d400000000014026c54',
'64f18b53000000001f007f97',
'64f186a0000000001e030350',
'64f17787000000001e0214d7',
'64f1651f000000001e02d519',
'64f1638b000000001e0316bb',
'64f15e5a000000001f038f8a',
'64f15d05000000001e020224',
'64f0c56e000000001f03a503',
'64f09480000000001f03ac7b',
'64f0901b000000001f0040c8',
'64f0818c000000001e022055',
'64f04a79000000001f006739',
'64f06319000000001e021264',
'64f05bad000000001e021ad2',
'64f0556d00000000140263c4',
'64f03e10000000001e030a7f',
'64eff9e5000000001e02a61a',
'64efd597000000001f00744a',
'64ef5315000000001e020786',
'64ef478e000000001e03e4f5',
'64ef3d93000000002000358d',
'64ef2d56000000001e030cb7',
'64ef05f2000000001f03aae6',
'64ef0534000000001f0379fe',
'64eed3c80000000020003d93',
'64ee9706000000001e02b8c8',
'64ee8669000000001e02e812',
'64ee1bed000000001e03f04e',
'64ee02110000000020000d14',
'64edfb16000000001e030011',
'64edef9e000000001e03ed50',
'64ede671000000001f00644b',
'64eddbf3000000001f00790a',
'64eddba0000000001e029713',
'64edda92000000001e02d006',
'64edd063000000001e03dbca',
'64ed6e33000000001d01560e',
'64ed3eb9000000001e02d34a',
'64ecd2f90000000020003d28',
'64ecc928000000001e032f73',
'64ecb420000000001e00f3ce',
'64eca805000000001e02c114',
'64eca297000000001f0363cd',
'64ec9a25000000001e00cfc5',
'64ec99f0000000001f0049e5',
'64ec891c000000001e0295cb',
'64ec6ebf000000000103db84',
'64ec6b2d0000000010033be0',
'64ec66cf00000000030228ea',
'64ec522d000000000b02b41b',
'64ec4d6c000000000301faef',
'64ec395e0000000010030003',
'64ec1fb9000000001701bc2c',
'64ebfd0000000000100322e2',
'64ebee46000000000800c04a',
'64ebe885000000001701bc78',
'64eb7a950000000003020878',
'64eb6584000000000b028ec4',
'64eb64fe000000000b028893',
'64eb4e32000000000301d897',
'64eb2dc50000000003023154',
'64eb136e000000000b02b376',
'64eb03a4000000001201c980',
'64eafefe0000000010030cf7',
'64eafbe0000000000b0280b3',
'64eae4ad0000000012019a62',
'64eac5940000000012019e2f',
'64eabe79000000000b02b6f8',
'64e88ea3000000000103cc74',
'64ea83350000000010030bdf',
'64ea35230000000010033f14',
'64ea25be0000000010031038',
'64ea192f0000000003020ef4',
'64ea0f900000000017018608',
'64e9b102000000000103e76a',
'64e9da44000000001700c534',
'64e9cf75000000000301d73e',
'64e9c994000000000b028931',
'64e98bfe000000000b02ac43',
'64e98ac20000000012019393',
'64e97124000000000103c74e',
'64e961220000000010033354',
'64e8ebdb000000000b02b4cf',
'64e8dad2000000000c034bab',
'64e8c2ab000000001201f6c6',
'64e8ae0e000000001700c79e',
'64e8a04d000000000800f21b',
'64e89d19000000000c037019',
'64e87b9d000000000800fc32',
'64e855cd0000000017018b03',
'64e8229d000000000b02bfc5',
'64e815a5000000001700dc3b',
'64e8084d000000001700e6d0',
'64e75d09000000000800eb53',
'64e75a5f000000000a01a1f3',
'64e75027000000001700ca9d',
'64e7487a000000000103cba6',
'64e71619000000001003122c',
'64e6e4180000000010031f95',
'64e6dac9000000001201a72c',
'64e6d9f3000000001700c64d',
'64e6cb52000000001201d738',
'64e6b40b000000001700f2d2',
'64e6b187000000000302140e',
'64e620a5000000001701925b',
'64e61d74000000000302120a',
'64e60b120000000010032473',
'64e60a42000000001201eecd',
'64e5fcee000000001700cdf5',
'64e5f93c00000000030201e1',
'64e59d4d0000000010030d6d',
'64e5d6e3000000000800ed75',
'64e5cee5000000001201c730',
'64e5a894000000000c035a4e',
'64e5b119000000000c035420',
'64e5904d000000000c037e3c',
'64e4d5e0000000000c03420e',
'64e58269000000000800fad9',
'64e503d90000000012018a0b',
'64e4d54a000000000a01a4ec',
'64e4c246000000001201d571',
'64e4a902000000000103dc1e',
'64e4a88a0000000003021116',
'64e49497000000000a0189f3',
'64e48b0a000000000a01928f',
'64e46cbb000000000c035b29',
'64e46830000000000103f063',
'64e460900000000003023fd3',
'64e457270000000012019560',
'64e4567b000000000301d9fb',
'64e43ce8000000000800e376',
'64e42f48000000001201facf',
'64e42ca8000000000c0374b2',
'64e3a962000000000800f648',
'64e38ac3000000001201e187',
'64e38484000000001700e4ba',
'64e37eae000000000301db7c',
'64e3728100000000030220e5',
'64e35df7000000001700fd81',
'64e3553d000000000800c30b',
'64e35211000000000103cbf7',
'64e30cfc000000001201c79b',
'64e307fa0000000010033a62',
'64e2f6dd000000001201d6f5',
'64e2f6b5000000000c037a0b',
'64e2e22f0000000012019bc5',
'64e2dc5d000000001201f8ce',
'64e2d7630000000003022c70',
'64e2bc5b00000000100326b9',
'64e28d23000000001201a1e6',
'64e26587000000000103cb35',
'64e226f4000000000103e60a',
'64e20cb1000000000302198f',
'64e207820000000010033952',
'64e202f4000000000301edee',
'64f1b72c000000001e030edf',
'64f1af3d000000001f039ffb',
'64f1ad84000000001f004fc3',
'64f19ec9000000001f039999',
'64f195b7000000001e00ca96',
'64f1935c000000001d015087',
'64f19207000000001f0078d1',
'64f18da6000000001302feff',
'64f186d7000000001e0306cd',
'64f16758000000001f03bd4e',
'64f15def000000001f03deb4',
'64f15d8b000000001e00eb8f',
'64f15c19000000001f0349d3',
'64f156a6000000001f03f8fb',
'64f154070000000020003459',
'64f15262000000001e023491',
'64f1522b000000001e03f39e',
'64f1516b00000000200026ed',
'64f148b60000000020003ae5',
'64f0e17e000000001e02be43',
'64f14191000000001f034cc3',
'64f1409e000000001f03eea0',
'64f1328900000000200020be',
'64f128c1000000001e033cc4',
'64f0da97000000001500adfb',
'64f0c6cf000000001500ac2f',
'64f0c5a9000000001d0150b6',
'64f0b709000000001d017583',
'64f0b55f000000001f039e5d',
'64f0a9af000000001303530b',
'64f0a91f0000000014026033',
'64f0a8c5000000001e03d32d',
'64f09fb9000000001d016a7b',
'64f09f15000000001e02e170',
'64f094db000000001e022d79',
'64f092cd000000001f004f12',
'64f08ed5000000001f034374',
'64f08a85000000001e021e3a',
'64f08a42000000001e02d480',
'64f08726000000001d017094',
'64f08447000000001e033bb5',
'64f082af000000001f03cb85',
'64f07db5000000001d0179cd',
'64f07940000000001d014ece',
'64f075b8000000001e022a9d',
'64f06efa000000001f00452e',
'64f06c9900000000140272db',
'64f0651e000000001e03fd98',
'64f06364000000001e02bed0',
'64f05e45000000001f0050a6',
'64f04c7b000000001f0040ca',
'64f0460a000000001302b56c',
'64f04427000000001e030a5d',
'64f03f97000000001e0315ee',
'64f03c5100000000200000b8',
'64f03822000000001f03628f',
'64f03344000000001e0215eb',
'64f020d7000000001e0331c8',
'64f0193d000000001d0140c0',
'64f0182b000000001e02d472',
'64f01707000000001e020952',
'64f015e6000000001e03201c',
'64f0117b000000001f0040b9',
'64f00be8000000001f037491',
'64f00087000000001e03e292',
'64effde1000000001e00e16e',
'64eff511000000001f03756b',
'64efee6e000000001f03eca4',
'64efb911000000001d016013',
'64ef8d77000000001e029c00',
'64ef7696000000001e02b995',
'64ef7582000000001f03ec13',
'64ef6507000000001f039114',
'64ef6365000000001f03e3f7',
'64ef54f8000000001e0313d1',
'64ef515b000000001e023d6a',
'64ef4e89000000001f034325',
'64ef48f8000000001e03247e',
'64ef4858000000001e031e02',
'64ef477a000000001f039d2f',
'64ef3eed000000001e03e0b0',
'64ef3402000000001e02cc26',
'64ef32bf000000001e029579',
'64ef2c09000000001e02c8a9',
'64ef2a8c0000000020001110',
'64ef29e6000000001f00536c',
'64ef25f2000000001f03effd',
'64ef22fe000000001e02fce3',
'64ef20fb000000001e033b34',
'64ef1e4d000000001f034416',
'64ef1606000000001e03cffd',
'64ef12dd000000001f0380fa',
'64ef0f8a000000001f0387c4',
'64ef09d5000000001f0396a6',
'64ef0480000000001e0302d8',
'64eef34c000000001f03fd8e',
'64eef20d000000001f007b92',
'64eee5f2000000001d01698e',
'64eee464000000001e0325ba',
'64eee04f000000001e029004',
'64eed610000000001e02fd1c',
'64eed48300000000200005d2',
'64eecd63000000001e02a38f',
'64eec9dc000000001f0395d2',
'64eec794000000001f03552a',
'64eec5a0000000002000121e',
'64eebd1d0000000020001638',
'64eebc82000000001d015b41',
'64eeb155000000001f03b8dd',
'64eeb096000000001e00cee1',
'64ee97f00000000020000fe9',
'64ee8f8b000000001e00ef36',
'64ee8855000000001e022fef',
'64ee4370000000001e022f33',
'64ee3f43000000001e00d73d',
'64ee2788000000001f00413d',
'64ee2449000000001e00e1dd',
'64ee1817000000001e03de4f',
'64ee14d5000000001f039211',
'64ee0764000000001e02866e',
'64ee06c3000000001f0344d8',
'64ee05df000000001e02d0ad',
'64ee0144000000001e00fefc',
'64edff53000000001e02d1de',
'64edf965000000001e02a62a',
'64edf36b0000000020001146',
'64eded7f000000001d015fbe',
'64ede608000000001e02e374',
'64ede59d000000001f03c7f1',
'64edde6f00000000200006bd',
'64edd6bb000000001e03fa3c',
'64edc2d9000000001f0068d9',
'64edb7dc000000001f03c1ae',
'64edb6f9000000001d017725',
'64edb54e000000001e02dd7a',
'64edb48d000000001e02ce24',
'64edad5e000000001e02abd7',
'64eda968000000001e032331',
'64eda89c000000001e03188f',
'64eda786000000001f03f5ac',
'64eda413000000001d014aa9',
'64eda392000000001f03778b',
'64ed9eb6000000001e0322c6',
'64ed96ce000000001e00c9b3',
'64ed92da0000000020002192',
'64ed921a000000001f00585e',
'64ed8cd5000000001e03d1ef',
'64ed81a9000000001f036c1c',
'64ed816a000000001e02e606',
'64ed7c31000000001f03e053',
'64ed73e2000000001f006a04',
'64ed6526000000001e03c6ff',
'64ed621d000000001f0358a5',
'64ed573a000000001e03d0de',
'64ed5485000000001e0334a7',
'64ed537c000000001e0223ff',
'64ed38ec000000001d016503',
'64ed2ead000000001e033005',
'64ecf1ef000000001f007240',
'64ece782000000001f005ac2',
'64ecde5b000000001e03f2e7',
'64ecc989000000001e021861',
'64ecc47e000000001f034d43',
'64ecbf59000000001e02a1a8',
'64ecbe56000000001e02d5f7',
'64ecb959000000001e02f433',
'64ecb8dd000000001f0394fd',
'64ecb759000000001e02e466',
'64ecafd5000000001e03cded',
'64ecae76000000001e02f20e',
'64ecaaf2000000001f0077fd',
'64eca233000000001f035ed4',
'64ec943900000000200037cd',
'64ec8ee3000000001e029c51',
'64ec8d85000000001f006d74',
'64ec84c2000000001e03fe61',
'64ec823d0000000020001019',
'64ec7a41000000001700cb55',
'64ec7807000000000a018885',
'64ec6d430000000012018339',
'64ec69e7000000001701bdab',
'64ec6846000000001201ad58',
'64ec6507000000001201f8bd',
'64ec63870000000012018900',
'64ec5ad100000000170197ea',
'64ec5504000000000a01896f',
'64ec54a8000000001201bf31',
'64ec4ddd000000001201dc10',
'64ec3a47000000000103c6b0',
'64ec2c01000000000b02a6a5',
'64ec2b65000000000103fce9',
'64ec24b20000000003022569',
'64ec19ff000000000c03793a',
'64ec157c0000000010032f60',
'64ec13840000000003022381',
'64ec0ca2000000000103cabd',
'64ebf0a4000000000b0280a8',
'64ebd119000000000a01a1d7',
'64eb75d0000000000a01b434',
'64eb7179000000000c03423f',
'64eb6e2d0000000010033432',
'64eb6b77000000000a018e14',
'64eb62570000000010030b1e',
'64eb60a0000000000301d22c',
'64eb603c000000001700c483',
'64eb5ebd000000000800de77',
'64eb4b3f0000000012018e4f',
'64eb2ded00000000030232cf',
'64eb273b000000001201aed0',
'64eb23bc0000000010033130',
'64eb1e1c000000001201a6b2',
'64eb1be6000000000301f38c',
'64eaf47e000000000301ca06',
'64eaefce000000000b0287a1',
'64ead664000000000a01901a',
'64ead229000000001201933b',
'64eacfc5000000001701abf0',
'64eacb870000000003020b8c',
'64eacac90000000010032121',
'64eac60f000000000c03759a',
'64ea9b00000000000c0340fb',
'64ea7796000000000c03496e',
'64ea75cc000000000b02b0d5',
'64ea26a1000000000800c914',
'64ea238c0000000017019353',
'64ea1711000000001201d24a',
'64ea12e8000000001701a8ec',
'64ea1202000000000a01a277',
'64ea116e0000000010031062',
'64ea0fca0000000017018872',
'64ea0f57000000001201b5d2',
'64ea0989000000000301d8aa',
'64ea01760000000010032c11',
'64ea015d000000000301e5ea',
'64ea01180000000010032575',
'64ea0046000000000c036516',
'64e9fe7a000000000a019dbe',
'64e9a9ff0000000017018c4d',
'64e9fd8e000000001700c0dd',
'64e9fc4b000000001201bc3e',
'64e9fc04000000000301fd27',
'64e9eee9000000000103d727',
'64e9ee57000000001201ee22',
'64e9ec0c000000000103c174',
'64e9ead8000000000b028a34',
'64e9ea550000000010030716',
'64e9e9b4000000001701a10b',
'64e9e654000000000103cd28',
'64e9d2ed000000000b028abc',
'64e9cee20000000012019ada',
'64e9c63d000000000103e94b',
'64e4f22e000000000a019bc8',
'64e9c0fc000000001700ec1d',
'64e9be63000000001700e668',
'64e9b728000000000c035b6d',
'64e9abf2000000001201ff42',
'64e9a6600000000017019e76',
'64e99f1b000000001701bc85',
'64e98d7f000000000800ec1a',
'64e98cc100000000030218f0',
'64e983c0000000000c036e5c',
'64e97ff3000000000a01b5ff',
'64e97ae3000000000a018788',
'64e9784000000000120192b0',
'64e9682b000000001701a72f',
'64e963ea000000000a01a4df',
'64e95e4c000000001701a059',
'64e31255000000000103f814',
'64e94aa2000000000103e558',
'64e94124000000001003275a',
'64e8f3eb000000000800f14c',
'64e8f3eb000000000800f14c',
'64e8f33a000000000b028127',
'64e8e7260000000012018134',
'64e8deaa000000001700e6eb',
'64e8db3a000000000800c341',
'64e8d739000000000103cc7c',
'64e8c9f6000000000c0350d3',
'64e8c9c0000000001201f035',
'64e8c7880000000003022695',
'64e8c4b8000000001201a984',
'64e8c131000000001201e658',
'64e8b9eb000000001201fb7f',
'64e8b3e7000000000b02822b',
'64e8a7cb00000000030223e5',
'64e8954c000000000c03461b',
'64e88377000000001700ddaa',
'64e87f90000000000c0347e2',
'64e8785500000000030220b6',
'64e857f7000000001700deb9',
'64e848d1000000000b02a538',
'64e82bab000000001700c68e',
'64e81a290000000003023adf',
'64e818d4000000000a01aa1d',
'64e80ffe000000000103fe83',
'64e80f2f000000001201885d',
'64e80be9000000001201cd34',
'64e794ba0000000017019f47',
'64e7742a000000001201e0c8',
'64e7734c000000001701a49b',
'64e766e4000000000b02a811',
'64e76671000000001701b6d0',
'64e75b020000000003020ff0',
'64e7580d000000001201dd96',
'64e74e39000000000103ff8a',
'64e74d54000000000b0292fc',
'64e73c11000000000c0342c1',
'64e73164000000000a01a081',
'64e72a4b0000000012018a0f',
'64e7242f000000000a01a60c',
'64e720b1000000000103c395',
'64e713d7000000000301f4f2',
'64e71294000000000301ea6b',
'64e7121e0000000012018e25',
'64e70ca0000000000103d117',
'64e709ee000000000301dc78',
'64e6f6ac000000001700e09d',
'64e6ea40000000001201de4a',
'64e6e78b000000000103e07c',
'64e6e4ec000000000c034985',
'64e6ddf8000000001201b71b',
'64e6d99b000000001700fee5',
'64e6d3fa00000000170191ef',
'64e6ce01000000001701b21e',
'64e6cd5d00000000030203ff',
'64e6ae460000000017018e34',
'64e68f6a00000000030203d9',
'64e68361000000001700c171',
'64e63892000000000c03569f',
'64e630d6000000000800e04a',
'64e6309d000000000b02b4a6',
'64e62c80000000001201d296',
'64e62c720000000010030520',
'64e62931000000000c034fdb',
'64e61fbf0000000017018b9e',
'64e60b5d0000000012019992',
'64e60a240000000003023e90',
'64e5f882000000001701be59',
'64e5ea67000000001701b49b',
'64e5e10d000000001700e039',
'64e5d983000000000800c406',
'64e5d7ce000000001201c6eb',
'64e5d245000000001700cf12',
'64e5d018000000000301e192',
'64e5cbde0000000010031497',
'64e5c8d1000000001201878f',
'64e5c852000000001201bd0a',
'64e5b7a60000000010030435',
'64e5b78d000000000a018720',
'64e5b657000000000c037da9',
'64e5b4f00000000003023ecd',
'64e5b267000000001700e354',
'64e5afff000000001201fcfa',
'64e5a86d000000001700ff22',
'64e5a4380000000012018ae4',
'64e5a27a00000000170180ae',
'64e5a002000000000800edc8',
'64e59ba2000000000c037178',
'64e59952000000001201a2be',
'64e5947d0000000010032644',
'64e591e1000000000a019742',
'64e590c2000000000103d739',
'64e58acf000000001700f574',
'64e587dc000000000302119a',
'64e58705000000000800e3ab',
'64e4efee000000001700e41a',
'64e582c50000000010032c12',
'64e57aad000000001201ee4f',
'64e56f32000000001700d235',
'64e5689f0000000012019f48',
'64e5671f000000000b029ac8',
'64e55e70000000000301c11d',
'64e547ff000000000800f308',
'64e535560000000017019a19',
'64e504ef0000000012018a7b',
'64e4e846000000000301ca70',
'64e4d536000000001201ce93',
'64e4c9a40000000003020c66',
'64e4b773000000000301d978',
'64e4a8630000000010033f5b',
'64e49e19000000000a01a2d0',
'64e493730000000003022e04',
'64e48b20000000000c034fa4',
'64e4867c000000001201f52e',
'64e485d30000000010030201',
'64e481ea000000000c0348db',
'64e47a54000000001700c71b',
'64e47621000000001201dc6f',
'64e470aa000000001700cca0',
'64e470330000000012019853',
'64e468cb000000000b029b68',
'64e46456000000001700eb7d',
'64e46300000000000800f04e',
'64e458110000000010033d88',
'64e45532000000000800e56b',
'64e4522c000000000a01bde5',
'64e45079000000000b02958b',
'64e44f7a0000000003023a0c',
'64e44aa5000000000301ee18',
'64e4449e000000000800ec14',
'64e44138000000000c0377e0',
'64e43753000000000301e9d2',
'64e43611000000001201bdf0',
'64e42e25000000001700c027',
'64e42dc2000000001201e5c9',
'64e42c84000000000103f1b7',
'64e4286d000000000800d240',
'64e4203f000000001003165e',
'64e3fedc0000000010033735',
'64e3f624000000000103d55a',
'64e3f259000000001700c3c2',
'64e3cf44000000000a01aca6',
'64e39bec000000000a01ae7f',
'64e38c2c000000000c034623',
'64e387bd000000000b02bda0',
'64e38605000000000a01be1a',
'64e384c80000000012019059',
'64e38042000000000a018b1d',
'64e37b590000000010030e38',
'64e36d84000000000b028b25',
'64e36ce4000000000c03599a',
'64e362b1000000000301f66a',
'64e36069000000000c036b29',
'64e35e9d000000001700c42c',
'64e35abd000000000a019bff',
'64e350e5000000000a01b1db',
'64e33c19000000000a0192c7',
'64e337d6000000000c035799',
'64e333c0000000000a018e1b',
'64e32b2b000000000301df4b',
'64e3241e000000000a018513',
'64e323ec000000001201a6bb',
'64e313c0000000001003062d',
'64e312a8000000000301e968',
'64e311a90000000010032ce7',
'64e301ef000000001700e4f2',
'64e2f292000000000b0287dd',
'64e2e8f10000000003023776',
'64e2e31a000000000a0189cf',
'64e2e15b00000000030225ae',
'64e167f1000000001700c28f',
'64e2deec000000000301c3dd',
'64e2dd9a000000000b02b7cb',
'64e2c9fb000000001201993e',
'64e2c877000000000b028c53',
'64e2c701000000001700c283',
'64e2b899000000000a018ee3',
'64e2ab92000000000a018dc4',
'64e295a7000000000800d8de',
'64e24e96000000000800e9a5',
'64e242b9000000000c03543d',
'64e23cce000000000301c7ad',
'64e237f7000000001201a036',
'64e225cc00000000100321be',
'64e23251000000000800c068',
'64e2297a000000000a01ba89',
'64e22113000000000c0371da',
'64e21f19000000000b02abfd',
'64e21e7c000000000a019c33',
'64e21105000000001701b140',
'64e20ac5000000000b0284fc',
'64e20a57000000001003209f',
'64e2093c0000000010030cca',
'64e2018f000000001700fa2c',
'64f1cba10000000020000e98',
'64ef2d56000000001e030cb7',
'64edd6c3000000001e03facf',
'64ecd2f90000000020003d28',
'64ecb420000000001e00f3ce',
'64ec6ebf000000000103db84',
'64ec7003000000000a018462',
'64ea93d50000000003021c7d',
'64e7f7d60000000010031b35',
'64e75027000000001700ca9d',
'64e4d5e0000000000c03420e',
'64e48b0a000000000a01928f',
'64e202f4000000000301edee',
'64ded834000000000302193a',
'64df37f0000000000c037086',
'64d9ec49000000000103faa5',
'64d7e04d0000000012018bca',
'64d7ab470000000010033ef9',
'64d76dac000000000302261c',
'64d42eca00000000170197e8',
'64d38bea000000000b0294ef',
'64d1d4f8000000000a019356',
'64d1d218000000001700cf21',
'64d1cbd6000000000b02b151',
'64d1cace000000001201d172',
'64d0884d0000000010033693',
'64cdae8b000000000c036e5b',
'64cd3545000000001700ef76',
'64cd22a1000000000b028116',
'64cccd44000000000b0298cc',
'64cca5d5000000000c03577e',
'64c96d620000000012018e59',
'64c8db95000000000800fa96',
'64c1186a000000000c035e79',
'64c0fd450000000010033ec5',
'64c0675300000000120199d3',
'64bf3ee900000000100305f4',
'64be354c000000000a0181e9',
'64f1ae36000000001e021133',
'64f16758000000001f03bd4e',
'64f128c1000000001e033cc4',
'64f0da97000000001500adfb',
'64f0cdd5000000001f03ddb7',
'64f0c5a9000000001d0150b6',
'64f0877a000000001d0173b5',
'64f05f70000000001f03b149',
'64f0460a000000001302b56c',
'64f03344000000001e0215eb',
'64f02f00000000001f004949',
'64f0193d000000001d0140c0',
'64f00be8000000001f037491',
'64eff0c2000000001e00dc8c',
'64ef477a000000001f039d2f',
'64ef32bf000000001e029579',
'64ef12dd000000001f0380fa',
'64eef20d000000001f007b92',
'64ee7c78000000001f03da37',
'64ee0c2d000000001d017cd7',
'64edf495000000001d015545',
'64ede608000000001e02e374',
'64edad5e000000001e02abd7',
'64eda89c000000001e03188f',
'64ed921a000000001f00585e',
'64ed6b0b000000001f034099',
'64ed621d000000001f0358a5',
'64ecd2420000000020003b3a',
'64ecae76000000001e02f20e',
'64ecaaf2000000001f0077fd',
'64ec8d73000000001f006b63',
'64ec2c01000000000b02a6a5',
'64ec2b65000000000103fce9',
'64ebf145000000000a019c54',
'64eb7179000000000c03423f',
'64eb6b030000000017019f23',
'64eb6660000000000800cedc',
'64eb60a0000000000301d22c',
'64eb1be6000000000301f38c',
'64eae7c300000000030216fb',
'64eada450000000010031d78',
'64ead33a000000001700dfd0',
'64ea75cc000000000b02b0d5',
'64ea1711000000001201d24a',
'64ea0fc50000000017018841',
'64e9ee57000000001201ee22',
'64e9e9b4000000001701a10b',
'64e9cee20000000012019ada',
'64e9c63d000000000103e94b',
'64e8ec17000000000b02b5cb',
'64e8b3e7000000000b02822b',
'64e71e9b000000000800fb83',
'64e8336d000000000103e19c',
'64e74fc0000000000a01a340',
'64e74e39000000000103ff8a',
'64e6d3fa00000000170191ef',
'64e68361000000001700c171',
'64e5f882000000001701be59',
'64e5d983000000000800c406',
'64e5b267000000001700e354',
'64e5ac46000000001700ee87',
'64e5a002000000000800edc8',
'64e587dc000000000302119a',
'64e54d3b00000000100303a9',
'64e547ff000000000800f308',
'64e4d536000000001201ce93',
'64e476c80000000003021ea9',
'64e49860000000000c03436c',
'64e4954b000000000800dfc5',
'64e48b20000000000c034fa4',
'64e3f624000000000103d55a',
'64e3cf44000000000a01aca6',
'64e39c8b0000000010033ed4',
'64e390c6000000000a01bf30',
'64e38e25000000000b028359',
'64e38605000000000a01be1a',
'64e35e9d000000001700c42c',
'64e323ec000000001201a6bb',
'64e2e923000000000800e073',
'64e24e96000000000800e9a5',
'64e225cc00000000100321be',
'64e1e8ba000000000800db1d',
'64e0e1200000000010032302',
'64e082ad000000001701afc1',
'64e08425000000000302001f',
'64e04c0f000000001201fd0d',
'64e0379c0000000003022239',
'64d861c000000000030230e8',
'64e01b6a0000000003023c11',
] 
    for note_id in note_id_list:
        try:
            handle_note(note_id)
        except TypeError as e:
            print(f"处理note_id {note_id} 出错: {e}")
            continue
            

if __name__ == '__main__':
    main()

cookie有效
用户: 维奇姐姐， 笔记标题: 接纳自己 豁然开朗♥️ 开始下载


FileNotFoundError: [Errno 2] No such file or directory: './datas/image/64f1c0ba000000001f03a3a6'

# final

In [14]:
import json
import os
import random
import re
import requests
import time
import execjs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import csv

base_url = 'https://www.xiaohongshu.com/user/profile/'
more_url = 'https://edith.xiaohongshu.com/api/sns/web/v1/user_posted'
feed_url = 'https://edith.xiaohongshu.com/api/sns/web/v1/feed'
img_url = 'https://sns-img-bd.xhscdn.com/'
video_url = 'https://sns-video-bd.xhscdn.com/'
js = execjs.compile(open(r'./info.js', 'r', encoding='utf-8').read())
if not os.path.exists("datas"):
    os.mkdir("datas")
class Bro:
    base_url = "https://www.xiaohongshu.com/user/profile/6185ce66000000001000705b"
    option = ChromeOptions()
    option.add_experimental_option('excludeSwitches', ['enable-automation'])
    # option.add_argument('--headless')
    # option.add_argument('--disable-gpu')
    option.add_argument("--disable-blink-features=AutomationControlled")
    option.add_argument('--start-maximized')
    option.add_argument('window-size=1920x1080')
    option.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36")
    s = Service(r"/Users/tianyc/Documents/python/xiaohongshu/Spider_XHS-master/chromedriver")

    def get_cookie(self):
        self.bro = webdriver.Chrome(service=self.s, options=self.option, )
        self.bro.implicitly_wait(10)
        self.bro.get(self.base_url)
        self.wait = WebDriverWait(self.bro, 3)
        try:
            me = self.wait.until(
                EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/div[1]/ul/li[4]/div/a')))
            print("已登录， 直接获取cookie")
        except:
            print("需要登录")
            time.sleep(1)
            print('正在点击主页登录按钮')
            login = self.wait.until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/div[2]/div[1]/ul/div[1]/button')))
            login.click()
            time.sleep(1)
            login_block = self.wait.until(EC.presence_of_element_located(
                (By.XPATH, '/html/body/div[1]/div[1]/div/div[1]')))
            while True:
                try:
                    time.sleep(5)
                    print('正在尝试登录')
                    if self.bro.current_url.find('https://www.xiaohongshu.com/') != -1:
                        if not login_block.is_displayed():
                            print('登录成功')
                    else:
                        continue
                except:
                    print('登录成功')
                    break

        print('正在获取cookie')
        time.sleep(5)
        cookies_ = self.bro.get_cookies()
        cookies = {}
        for cookie in cookies_:
            cookies[cookie['name']] = cookie['value']
        self.bro.quit()
        with open("cookies.txt", "w", encoding="utf-8") as f:
            f.write(str(cookies))
        return cookies
        
def download_media(path, name, img_url_id, type, fm=False):
    media_url = img_url + img_url_id if type == 'image' else video_url + img_url_id
    if type == 'image':
        print("封面开始下载") if fm else print(f"第{name}个图片开始下载")
        content = requests.get(media_url).content
        with open(path + '/' + name + '.jpg', mode="wb") as f:
            f.write(content)
            print("封面下载完成") if fm else print(f"第{name}个图片下载完成")
    elif type == 'video':
        # 将视频URL保存到CSV文件中
        csv_file_path = path + '/videos.csv'
        with open(csv_file_path, mode='a', encoding='utf-8', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([name, media_url])
        print(f"{name}保存到CSV文件中")   

def timestamp_to_time(timestamp):
    time_local = time.localtime(timestamp / 1000)
    dt = time.strftime("%Y年%m月%d日%H点%M分%S秒", time_local)
    return dt

def checkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

def save_detail(path, detail):
    with open(path + '/' + 'detail.txt', mode="w", encoding="utf-8") as f:
        # 逐行输出到txt里
        f.write(f"用户名: {detail['user_nickname']}\n")
        f.write(f"笔记标题: {detail['note_title']}\n")
        f.write(f"笔记描述: {detail['note_desc']}\n")
        f.write(f"笔记时间: {detail['note_time']}\n")
        f.write(f"笔记地点: {detail['note_ipLocation']}\n")

cookies = {
    "xsecappid": "",
    "a1": "",
    "webId": "",
    "gid": "",
    "webBuild": "3.3.4",
    "web_session": "",
    "websectiga": "",
    "sec_poison_id": ""
}
headers = {
    "authority": "edith.xiaohongshu.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "content-type": "application/json;charset=UTF-8",
    "origin": "https://www.xiaohongshu.com",
    "referer": "https://www.xiaohongshu.com/",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.188",
    "x-s": "",
    "x-t": ""
}
params = {
    "num": "30",
    "cursor": "",
    "user_id": "",
    "image_scenes": ""
}

def check_cookie():
    global cookies
    try:
        if not os.path.exists("cookies.txt"):
            raise Exception("获取cookie")
        test_user_id = '5ad2ede14eacab146f865fe9'
        with open("cookies.txt", "r", encoding="utf-8") as f:
            cookies_obj = f.read()
        cookies_local = eval(cookies_obj)
        params['user_id'] = test_user_id
        params['cursor'] = ''
        api = f"/api/sns/web/v1/user_posted?num=30&cursor=&user_id={test_user_id}&image_scenes="
        ret = js.call('get_xs', api, '', cookies_local['a1'])
        headers['x-s'], headers['x-t'] = ret['X-s'], str(ret['X-t'])
        response = requests.get(more_url, headers=headers, cookies=cookies_local, params=params)
        res = response.json()
        if not res["success"]:
            raise Exception("cookie失效")
        else:
            cookies = cookies_local
            print("cookie有效")
    except:
        print("cookie失效，正在重新获取")
        bro = Bro()
        cookies_local = bro.get_cookie()
        if cookies_local:
            print('cookies获取成功')
        else:
            print('cookies获取失败')
        cookies = cookies_local
def handle_note(note_id):
    data = {
        "source_note_id": note_id
    }
    data = json.dumps(data, separators=(',', ':'))
    ret = js.call('get_xs', '/api/sns/web/v1/feed', note_id, cookies['a1'])
    headers['x-s'], headers['x-t'] = ret['X-s'], str(ret['X-t'])
    response = requests.post(feed_url, headers=headers, cookies=cookies, data=data)
    res = response.json()
    data = res['data']['items'][0]
    user_dir_name = re.sub(r'[\\/:*?"<>|]', '', data['note_card']['user']['nickname'])
    user = {
        "nick_name": data['note_card']['user']['nickname'],
        "user_id": data['note_card']['user']['user_id'],
        "avatar": data['note_card']['user']['avatar'],
        "user_dir_name": user_dir_name
    }
    note_title = data['note_card']['title']
    print(f'用户: {user["nick_name"]}， 笔记标题: {note_title} 开始下载')
    note_type = data['note_card']['type']
    note_time = data['note_card']['time']
    if 'ip_location' in data['note_card']:
        note_ip_location = data['note_card']['ip_location']
    else:
        note_ip_location = '未知'
    note_desc = data['note_card']['desc']
    note_file_name = re.sub(r'[\\/:*?"<>|]', '', note_title)
    file_path = f"./datas1/{note_type}/{note_id}"
    checkdir(file_path)

    if note_type == 'normal':
        for index, img in enumerate(data['note_card']['image_list']):
            img_url_id = img['trace_id']
            download_media(file_path, str(index), img_url_id, 'image')

    elif note_type == 'video':
        img_url_id = data['note_card']['image_list'][0]['trace_id']
        download_media(file_path, '封面', img_url_id, 'image', fm=True)
        video_url_id = data['note_card']['video']['consumer']['origin_video_key']
        download_media(file_path, '视频', video_url_id, 'video')
    detail = {
        "user_nickname": user['nick_name'],
        "note_title": note_title,
        "note_desc": note_desc,
        "note_time": timestamp_to_time(int(note_time)),
        "note_ipLocation": note_ip_location,

    }
    save_detail(file_path, detail)
    print(f'用户: {user["nick_name"]}， 笔记标题: {note_title} 下载完成')
    print('--------------------------------------------------------------------------------')

def main():
    check_cookie()
    # https://www.xiaohongshu.com/explore/           64d06670000000000800fb4a    详细页获取
    note_id_list = [

'62bfc2620000000009038637',

    ]
    for note_id in note_id_list:
        try:
            handle_note(note_id)
        except TypeError as e:
            print(f"处理note_id {note_id} 出错: {e}")
            continue
            



if __name__ == '__main__':
    main()

cookie有效
用户: Z先生笔记， 笔记标题: 西班牙大流感 开始下载
第0个图片开始下载
第0个图片下载完成
第1个图片开始下载
第1个图片下载完成
第2个图片开始下载
第2个图片下载完成
用户: Z先生笔记， 笔记标题: 西班牙大流感 下载完成
--------------------------------------------------------------------------------
